# Chapter 03: Maximum-Entropy RL — Langevin Dynamics + Maximum Caliber

> "SAC is not inspired by physics — it is physics applied to control."

This notebook demonstrates the deep connection between statistical physics and reinforcement learning through:
1. **Langevin Dynamics**: Stochastic exploration in energy landscapes
2. **Boltzmann Policies**: The optimal policy form from MaxEnt principle
3. **Soft Actor-Critic (SAC)**: State-of-the-art MaxEnt (Maximum Entropy) RL algorithm


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal
from collections import deque
import random
import gymnasium as gym

# Reproducibility
np.random.seed(42)
torch.manual_seed(42)
random.seed(42)

# Plot styling
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## Part 1: Langevin Dynamics — Exploring Energy Landscapes

### Theory

Langevin dynamics describes a particle in a potential $V(x)$ with thermal fluctuations:

$$
\dot{x} = -\nabla V(x) + \sqrt{2D} \, \eta(t)
$$

The stationary distribution is the Boltzmann distribution:

$$
p_{\text{eq}}(x) \propto \exp\left(-\frac{V(x)}{k_B T}\right)
$$

**Temperature controls exploration**: High $T$ → wide exploration; Low $T$ → exploitation.

In [ ]:
def double_well_potential(x):
    """Double-well potential: V(x) = (x^2 - 1)^2"""
    return (x**2 - 1)**2

def gradient_potential(x):
    """Gradient of double-well potential"""
    return 4 * x * (x**2 - 1)

def langevin_dynamics(V_grad, x0, T, dt=0.01, steps=10000):
    """
    Simulate Langevin dynamics.
    
    Args:
        V_grad: Gradient of potential function
        x0: Initial position
        T: Temperature (controls exploration)
        dt: Time step
        steps: Number of steps
    
    Returns:
        trajectory: Array of positions over time
    """
    D = T  # Diffusion coefficient (set k_B = gamma = 1)
    trajectory = np.zeros(steps)
    x = x0
    
    for i in range(steps):
        # Langevin update: drift + diffusion
        drift = -V_grad(x)
        diffusion = np.sqrt(2 * D * dt) * np.random.randn()
        x = x + drift * dt + diffusion
        trajectory[i] = x
    
    return trajectory

# Simulate at different temperatures
temperatures = [0.1, 0.5, 1.0, 2.0]
trajectories = {}

for T in temperatures:
    traj = langevin_dynamics(gradient_potential, x0=0.5, T=T, steps=50000)
    trajectories[T] = traj

In [ ]:
# Visualize Langevin dynamics
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

x_range = np.linspace(-2, 2, 200)
V = double_well_potential(x_range)

for idx, T in enumerate(temperatures):
    ax = axes[idx]
    
    # Plot potential landscape
    ax2 = ax.twinx()
    ax2.plot(x_range, V, 'k-', alpha=0.3, linewidth=2, label='Potential V(x)')
    ax2.set_ylabel('Potential V(x)', fontsize=11)
    ax2.set_ylim(-0.5, 3)
    
    # Plot histogram of visited states
    traj = trajectories[T]
    ax.hist(traj[10000:], bins=100, density=True, alpha=0.6, color='blue', label='Empirical')
    
    # Theoretical Boltzmann distribution
    Z = np.trapezoid(np.exp(-V / T), x_range)
    p_boltzmann = np.exp(-V / T) / Z
    ax.plot(x_range, p_boltzmann, 'r-', linewidth=2, label='Boltzmann (theory)')
    
    ax.set_xlabel('Position x', fontsize=11)
    ax.set_ylabel('Probability Density', fontsize=11)
    ax.set_title(f'Temperature T = {T}', fontsize=12, fontweight='bold')
    ax.legend(loc='upper left', fontsize=9)
    ax.set_xlim(-2, 2)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Langevin Dynamics: Temperature Controls Exploration', 
             fontsize=14, fontweight='bold', y=1.02)
plt.show()

print("Key Observations:")
print(f"• T={temperatures[0]}: Low temperature → trapped in one well (exploitation)")
print(f"• T={temperatures[-1]}: High temperature → explores both wells (exploration)")
print(f"• Empirical histograms match theoretical Boltzmann distribution!")

## Part 2: Boltzmann Policy — The Optimal Policy Form

### Theory

In MaxEnt RL, the optimal policy is:

$$
\pi^*(a|s) = \frac{1}{Z(s)} \exp\left(\frac{Q^*(s,a)}{\alpha}\right)
$$

This is **exactly the Boltzmann distribution** with:
- "Energy" = $-Q^*(s,a)$ (negative Q-value)
- "Temperature" = $\alpha$ (entropy coefficient)

**This is not a heuristic** — it's the unique solution to:
$$
\max_\pi \, \mathbb{E}_\pi[Q(s, \cdot)] + \alpha H(\pi)
$$

In [ ]:
def boltzmann_policy(Q_values, alpha):
    """
    Compute Boltzmann policy from Q-values.
    
    Args:
        Q_values: Array of Q(s, a) for each action
        alpha: Temperature parameter
    
    Returns:
        policy: Probability distribution over actions
    """
    exp_Q = np.exp(Q_values / alpha)
    Z = np.sum(exp_Q)  # Partition function
    return exp_Q / Z

def policy_entropy(policy):
    """Compute entropy H(π) = -Σ π(a) log π(a)"""
    return -np.sum(policy * np.log(policy + 1e-10))

# Example: 5 actions with different Q-values
Q_values = np.array([1.0, 3.0, 2.0, 0.5, 2.5])
actions = np.arange(len(Q_values))
alphas = [0.1, 0.5, 1.0, 2.0, 5.0]

# Compute policies at different temperatures
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for idx, alpha in enumerate(alphas):
    policy = boltzmann_policy(Q_values, alpha)
    entropy = policy_entropy(policy)
    expected_Q = np.sum(policy * Q_values)
    
    ax = axes[idx]
    bars = ax.bar(actions, policy, alpha=0.7, color='steelblue', edgecolor='black')
    
    # Highlight best action
    best_action = np.argmax(Q_values)
    bars[best_action].set_color('crimson')
    bars[best_action].set_alpha(0.9)
    
    ax.set_xlabel('Action', fontsize=11)
    ax.set_ylabel('Probability π(a|s)', fontsize=11)
    ax.set_title(f'α = {alpha}\nH(π) = {entropy:.2f}, E[Q] = {expected_Q:.2f}', 
                 fontsize=11, fontweight='bold')
    ax.set_ylim(0, 1)
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add Q-values as text
    for i, (a, q) in enumerate(zip(actions, Q_values)):
        ax.text(a, -0.08, f'Q={q:.1f}', ha='center', fontsize=9, color='gray')

# Remove extra subplot
fig.delaxes(axes[-1])

plt.tight_layout()
plt.suptitle('Boltzmann Policy: Temperature Controls Exploration-Exploitation', 
             fontsize=14, fontweight='bold', y=1.02)
plt.show()

print("\nKey Observations:")
print(f"• α→0: Policy becomes deterministic (picks best action, red bar)")
print(f"• α→∞: Policy becomes uniform (maximum exploration)")
print(f"• Intermediate α: Balances exploitation (high Q) and exploration (entropy)")

## Part 3: Soft Actor-Critic (SAC) Implementation

### Algorithm Overview

SAC maintains:
1. **Soft Q-networks**: $Q_\phi(s, a)$ (two networks, take minimum)
2. **Policy network**: $\pi_\theta(a|s)$ (Gaussian for continuous actions)
3. **Target networks**: $Q_{\phi'}$ (slowly updated)

**Key equations**:
- Soft Bellman target: $y = r + \gamma (Q_{\text{min}}(s', a') - \alpha \log \pi(a'|s'))$
- Policy loss: $L_\pi = \mathbb{E}[\alpha \log \pi(a|s) - Q(s, a)]$

In [ ]:
# Neural network architectures

class QNetwork(nn.Module):
    """Soft Q-function approximator."""
    def __init__(self, state_dim, action_dim, hidden_dim=256):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim + action_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
    
    def forward(self, state, action):
        x = torch.cat([state, action], dim=-1)
        return self.net(x)


class GaussianPolicy(nn.Module):
    """Squashed Gaussian policy for continuous actions."""
    def __init__(self, state_dim, action_dim, hidden_dim=256, log_std_min=-20, log_std_max=2):
        super().__init__()
        self.log_std_min = log_std_min
        self.log_std_max = log_std_max
        
        self.trunk = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        
        self.mean_head = nn.Linear(hidden_dim, action_dim)
        self.log_std_head = nn.Linear(hidden_dim, action_dim)
    
    def forward(self, state):
        x = self.trunk(state)
        mean = self.mean_head(x)
        log_std = self.log_std_head(x)
        log_std = torch.clamp(log_std, self.log_std_min, self.log_std_max)
        return mean, log_std
    
    def sample(self, state, deterministic=False):
        """
        Sample action from policy.
        
        Returns:
            action: Sampled action (squashed to [-1, 1])
            log_prob: Log probability of the action
        """
        mean, log_std = self.forward(state)
        std = log_std.exp()
        
        if deterministic:
            # Use mean for evaluation
            action = torch.tanh(mean)
            return action, None
        
        # Reparameterization trick
        normal = Normal(mean, std)
        x = normal.rsample()  # Sample with reparameterization
        action = torch.tanh(x)
        
        # Compute log probability with tanh correction
        log_prob = normal.log_prob(x)
        log_prob -= torch.log(1 - action.pow(2) + 1e-6)
        log_prob = log_prob.sum(dim=-1, keepdim=True)
        
        return action, log_prob


class ReplayBuffer:
    """Experience replay buffer."""
    def __init__(self, capacity=1000000):
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        
        return (
            torch.from_numpy(np.array(states)).float(),
            torch.from_numpy(np.array(actions)).float(),
            torch.from_numpy(np.array(rewards)).float().unsqueeze(1),
            torch.from_numpy(np.array(next_states)).float(),
            torch.from_numpy(np.array(dones)).float().unsqueeze(1)
        )
    
    def __len__(self):
        return len(self.buffer)



In [ ]:
class SAC:
    """Soft Actor-Critic agent."""
    def __init__(self, state_dim, action_dim, alpha=0.2, gamma=0.99, tau=0.005, lr=3e-4):
        self.gamma = gamma
        self.tau = tau
        self.alpha = alpha
        
        # Networks
        self.q1 = QNetwork(state_dim, action_dim)
        self.q2 = QNetwork(state_dim, action_dim)
        self.q1_target = QNetwork(state_dim, action_dim)
        self.q2_target = QNetwork(state_dim, action_dim)
        
        # Copy parameters to targets
        self.q1_target.load_state_dict(self.q1.state_dict())
        self.q2_target.load_state_dict(self.q2.state_dict())
        
        self.policy = GaussianPolicy(state_dim, action_dim)
        
        # Optimizers
        self.q_optimizer = torch.optim.Adam(
            list(self.q1.parameters()) + list(self.q2.parameters()), lr=lr
        )
        self.policy_optimizer = torch.optim.Adam(self.policy.parameters(), lr=lr)
        
        # Metrics
        self.q_losses = []
        self.policy_losses = []
        self.entropies = []
    
    def select_action(self, state, deterministic=False):
        """Select action from policy."""
        with torch.no_grad():
            state = torch.FloatTensor(state).unsqueeze(0)
            action, _ = self.policy.sample(state, deterministic=deterministic)
        return action.squeeze(0).numpy()
    
    def update(self, batch):
        """Update networks with a batch of experiences."""
        states, actions, rewards, next_states, dones = batch
        
        # --- Critic Update ---
        with torch.no_grad():
            next_actions, next_log_probs = self.policy.sample(next_states)
            q1_next = self.q1_target(next_states, next_actions)
            q2_next = self.q2_target(next_states, next_actions)
            q_next = torch.min(q1_next, q2_next)
            
            # Soft Bellman target
            target = rewards + self.gamma * (1 - dones) * (q_next - self.alpha * next_log_probs)
        
        q1_pred = self.q1(states, actions)
        q2_pred = self.q2(states, actions)
        
        q1_loss = F.mse_loss(q1_pred, target)
        q2_loss = F.mse_loss(q2_pred, target)
        q_loss = q1_loss + q2_loss
        
        self.q_optimizer.zero_grad()
        q_loss.backward()
        self.q_optimizer.step()
        
        # --- Actor Update ---
        new_actions, log_probs = self.policy.sample(states)
        q1_new = self.q1(states, new_actions)
        q2_new = self.q2(states, new_actions)
        q_new = torch.min(q1_new, q2_new)
        
        policy_loss = (self.alpha * log_probs - q_new).mean()
        
        self.policy_optimizer.zero_grad()
        policy_loss.backward()
        self.policy_optimizer.step()
        
        # --- Soft Update Target Networks ---
        self._soft_update(self.q1, self.q1_target)
        self._soft_update(self.q2, self.q2_target)
        
        # Record metrics
        self.q_losses.append(q_loss.item())
        self.policy_losses.append(policy_loss.item())
        self.entropies.append(-log_probs.mean().item())
    
    def _soft_update(self, source, target):
        """Polyak averaging: θ' ← τθ + (1-τ)θ'"""
        for param, target_param in zip(source.parameters(), target.parameters()):
            target_param.data.copy_(
                self.tau * param.data + (1 - self.tau) * target_param.data
            )


print("✓ SAC agent implemented")

## Part 4: Training on Pendulum Environment

We'll train SAC on the classic **Pendulum-v1** environment from Gymnasium:
- **State**: (cos(θ), sin(θ), angular velocity)
- **Action**: Torque [-2, 2]
- **Goal**: Swing up and balance the pendulum

This demonstrates MaxEnt RL on a continuous control task.

In [ ]:
def train_sac(env_name='Pendulum-v1', num_episodes=200, batch_size=256, alpha=0.2):
    """
    Train SAC on a Gymnasium environment.
    
    Args:
        env_name: Name of the Gymnasium environment
        num_episodes: Number of training episodes
        batch_size: Batch size for updates
        alpha: Temperature parameter (entropy coefficient)
    
    Returns:
        agent: Trained SAC agent
        rewards: List of episode rewards
        entropies: List of average entropies per episode
    """
    env = gym.make(env_name)
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.shape[0]
    
    agent = SAC(state_dim, action_dim, alpha=alpha)
    replay_buffer = ReplayBuffer()
    
    episode_rewards = []
    episode_entropies = []
    
    for episode in range(num_episodes):
        state, _ = env.reset()
        episode_reward = 0
        episode_entropy = []
        done = False
        
        while not done:
            # Select action
            action = agent.select_action(state)
            
            # Take step in environment
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            
            # Store transition
            replay_buffer.push(state, action, reward, next_state, float(done))
            
            episode_reward += reward
            state = next_state
            
            # Update agent
            if len(replay_buffer) > batch_size:
                batch = replay_buffer.sample(batch_size)
                agent.update(batch)
                if agent.entropies:
                    episode_entropy.append(agent.entropies[-1])
        
        episode_rewards.append(episode_reward)
        episode_entropies.append(np.mean(episode_entropy) if episode_entropy else 0)
        
        if (episode + 1) % 20 == 0:
            avg_reward = np.mean(episode_rewards[-20:])
            avg_entropy = np.mean(episode_entropies[-20:])
            print(f"Episode {episode+1}/{num_episodes} | "
                  f"Avg Reward: {avg_reward:.2f} | "
                  f"Avg Entropy: {avg_entropy:.2f}")
    
    env.close()
    return agent, episode_rewards, episode_entropies


# Train SAC
print("Training SAC on Pendulum-v1...\n")
agent, rewards, entropies = train_sac(num_episodes=200, alpha=0.2)

In [ ]:
# Visualize training progress
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# Episode rewards
ax = axes[0]
ax.plot(rewards, alpha=0.3, color='blue')
window = 10
smoothed = np.convolve(rewards, np.ones(window)/window, mode='valid')
ax.plot(range(window-1, len(rewards)), smoothed, color='darkblue', linewidth=2, label='Smoothed')
ax.set_xlabel('Episode', fontsize=11)
ax.set_ylabel('Episode Reward', fontsize=11)
ax.set_title('Learning Curve', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()

# Policy entropy
ax = axes[1]
ax.plot(entropies, alpha=0.6, color='orange', linewidth=1.5)
ax.axhline(y=np.mean(entropies), color='red', linestyle='--', linewidth=2, label='Mean')
ax.set_xlabel('Episode', fontsize=11)
ax.set_ylabel('Policy Entropy H(π)', fontsize=11)
ax.set_title('Exploration (Entropy Over Time)', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()

# Q-loss and policy loss
ax = axes[2]
if agent.q_losses:
    window = 100
    q_smoothed = np.convolve(agent.q_losses, np.ones(window)/window, mode='valid')
    p_smoothed = np.convolve(agent.policy_losses, np.ones(window)/window, mode='valid')
    
    ax.plot(q_smoothed, color='green', linewidth=2, label='Q-loss')
    ax.plot(p_smoothed, color='purple', linewidth=2, label='Policy loss')
    ax.set_xlabel('Update Step', fontsize=11)
    ax.set_ylabel('Loss', fontsize=11)
    ax.set_title('Training Losses', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal Performance:")
print(f"• Average reward (last 20 episodes): {np.mean(rewards[-20:]):.2f}")
print(f"• Average entropy (last 20 episodes): {np.mean(entropies[-20:]):.2f}")
print(f"• Policy maintains exploration throughout training!")

## Part 5: Temperature Ablation Study

Let's empirically verify that the temperature parameter $\alpha$ controls the exploration-exploitation tradeoff.

In [ ]:
# Train SAC with different temperatures
alphas = [0.05, 0.1, 0.2, 0.5]
results = {}

print("Running temperature ablation study...\n")
for alpha in alphas:
    print(f"Training with α = {alpha}...")
    agent_i, rewards_i, entropies_i = train_sac(
        num_episodes=500, 
        alpha=alpha, 
        batch_size=256
    )
    results[alpha] = {
        'agent': agent_i,
        'rewards': rewards_i,
        'entropies': entropies_i
    }
    print()

In [ ]:
# Visualize temperature effects
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = ['red', 'orange', 'green', 'blue']
window = 10

# Rewards
ax = axes[0]
for (alpha, data), color in zip(results.items(), colors):
    rewards = data['rewards']
    smoothed = np.convolve(rewards, np.ones(window)/window, mode='valid')
    ax.plot(range(window-1, len(rewards)), smoothed, 
            color=color, linewidth=2, label=f'α = {alpha}')

ax.set_xlabel('Episode', fontsize=11)
ax.set_ylabel('Episode Reward (smoothed)', fontsize=11)
ax.set_title('Performance vs Temperature', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Entropies
ax = axes[1]
for (alpha, data), color in zip(results.items(), colors):
    entropies = data['entropies']
    ax.plot(entropies, color=color, linewidth=2, alpha=0.7, label=f'α = {alpha}')

ax.set_xlabel('Episode', fontsize=11)
ax.set_ylabel('Policy Entropy H(π)', fontsize=11)
ax.set_title('Exploration vs Temperature', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Observations:")
print("• Higher α → Higher entropy (more exploration)")
print("• Lower α → Faster convergence but risk of local optima")
print("• Optimal α balances exploration and exploitation")
print(f"\nFinal rewards (last 20 episodes):")
for alpha, data in results.items():
    final_reward = np.mean(data['rewards'][-20:])
    print(f"  α = {alpha}: {final_reward:.2f}")

## Part 6: Visualizing the Trained Policy

Let's visualize how the trained policy behaves in the state space.

In [ ]:
def evaluate_policy(agent, env_name='Pendulum-v1', num_episodes=5, render=False):
    """Evaluate trained policy."""
    env = gym.make(env_name, render_mode='rgb_array' if render else None)
    rewards = []
    
    for ep in range(num_episodes):
        state, _ = env.reset()
        episode_reward = 0
        done = False
        
        while not done:
            action = agent.select_action(state, deterministic=True)
            state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            episode_reward += reward
        
        rewards.append(episode_reward)
        print(f"Episode {ep+1}: Reward = {episode_reward:.2f}")
    
    env.close()
    print(f"\nAverage reward: {np.mean(rewards):.2f} ± {np.std(rewards):.2f}")
    return rewards

# Evaluate best agent
print("Evaluating trained policy...\n")
eval_rewards = evaluate_policy(agent, num_episodes=5)

In [ ]:
# Visualize policy and Q-function in state space
def visualize_policy_and_Q(agent, resolution=50):
    """Visualize policy actions and Q-values over state space."""
    # Sample states: vary angle and angular velocity
    angles = np.linspace(-np.pi, np.pi, resolution)
    velocities = np.linspace(-8, 8, resolution)
    
    actions_grid = np.zeros((resolution, resolution))
    q_values_grid = np.zeros((resolution, resolution))
    
    for i, theta in enumerate(angles):
        for j, vel in enumerate(velocities):
            state = np.array([np.cos(theta), np.sin(theta), vel])
            
            # Get action
            action = agent.select_action(state, deterministic=True)
            actions_grid[j, i] = action[0]
            
            # Get Q-value
            with torch.no_grad():
                state_t = torch.FloatTensor(state).unsqueeze(0)
                action_t = torch.FloatTensor(action).unsqueeze(0)
                q1 = agent.q1(state_t, action_t).item()
                q2 = agent.q2(state_t, action_t).item()
                q_values_grid[j, i] = min(q1, q2)
    
    # Plot
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Policy actions
    ax = axes[0]
    im1 = ax.imshow(actions_grid, extent=[-np.pi, np.pi, -8, 8], 
                    aspect='auto', origin='lower', cmap='RdBu_r')
    ax.set_xlabel('Angle θ (rad)', fontsize=11)
    ax.set_ylabel('Angular Velocity (rad/s)', fontsize=11)
    ax.set_title('Policy Actions π(s)', fontsize=12, fontweight='bold')
    ax.axvline(x=0, color='black', linestyle='--', alpha=0.5, label='Upright')
    plt.colorbar(im1, ax=ax, label='Torque')
    
    # Q-values
    ax = axes[1]
    im2 = ax.imshow(q_values_grid, extent=[-np.pi, np.pi, -8, 8], 
                    aspect='auto', origin='lower', cmap='viridis')
    ax.set_xlabel('Angle θ (rad)', fontsize=11)
    ax.set_ylabel('Angular Velocity (rad/s)', fontsize=11)
    ax.set_title('Q-Values Q(s, π(s))', fontsize=12, fontweight='bold')
    ax.axvline(x=0, color='white', linestyle='--', alpha=0.5, label='Upright')
    plt.colorbar(im2, ax=ax, label='Q-value')
    
    plt.tight_layout()
    plt.show()
    
    print("Interpretation:")
    print("• Left plot: Policy applies torque to swing up and stabilize")
    print("• Right plot: Q-values are highest near upright position (θ=0)")
    print("• Policy learns smooth, continuous control strategy")

visualize_policy_and_Q(agent, resolution=40)